<a href="https://colab.research.google.com/github/naimadmpi10/video-subtitle-/blob/main/ArabicToBanglaSubtitle8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Step 2: প্রয়োজনীয় লাইব্রেরি ইনস্টল করুন
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q deep-translator

In [ ]:

# Step 3: ভিডিও ফাইলের পথ দিন (আপনার ভিডিও Drive-এ আপলোড করা থাকতে হবে)
video_path = "/content/drive/MyDrive/Videos/as.mp4"  # এখানে ফাইলের নাম ও path দিন
!apt-get update && apt-get install -y ffmpeg

In [ ]:

# Step 4: ভিডিও থেকে ট্রান্সক্রিপশন করুন
import whisper

model = whisper.load_model("base")
result = model.transcribe(video_path)
segments = result['segments']

In [11]:
from deep_translator import GoogleTranslator
translated_segments = []
for seg in result['segments']:
    bn_text = GoogleTranslator(source='ar', target='bn').translate(seg['text'])
    translated_segments.append((seg['start'], seg['end'], bn_text))
    bn_text = GoogleTranslator(source='ar', target='bn').translate(seg['text'])
    translated_segments.append((seg['start'], seg['end'], bn_text))

In [ ]:
!apt install fonts-noto

In [20]:
def format_ass_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds % 1) * 100)
    return f"{hours}:{minutes:02}:{secs:02}.{millis:02}"

# ইউজার ইনপুট
full_text = input("তোমার বাংলা ডায়লগগুলো লিখো (এক লাইনে বা একাধিক লাইনে):\n").strip()
video_duration_input = input("ভিডিওর মোট দৈর্ঘ্য কত সেকেন্ড? (শুধু সংখ্যা):\n")

if not full_text:
    raise ValueError("ডায়লগ খালি দেওয়া হয়েছে!")

try:
    video_duration = float(video_duration_input)
except:
    raise ValueError("ভিডিওর সময় সংখ্যায় দিন।")

if video_duration <= 0:
    raise ValueError("ভিডিওর দৈর্ঘ্য ০ বা ঋণাত্মক হতে পারে না!")

# ইনপুট ক্লিন করা
merged_text = " ".join(full_text.splitlines()).strip()

# একাধিক স্প্লিটার ব্যবহার করে ভাগ করা
import re
raw_segments = re.split(r"।|,|এবং", merged_text)

# ডায়লগ ফাইনালভাবে সাজানো
segments = [seg.strip() for seg in raw_segments if seg.strip()]
segment_count = len(segments)

if segment_count == 0:
    raise ValueError("একটি বাক্যও পাওয়া যায়নি।")

segment_duration = video_duration / segment_count

translated_segments = []
for i, sentence in enumerate(segments):
    start_time = i * segment_duration
    end_time = (i + 1) * segment_duration
    if not sentence.endswith("।"):
        sentence += "।"
    translated_segments.append((start_time, end_time, sentence))

# .ASS ফাইল তৈরি
ass_header = """
[Script Info]
Title: Bengali Subtitles
ScriptType: v4.00+
PlayResY: 720
PlayResX: 1280
Timer: 100.0000

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, BackColour, OutlineColour, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,Noto Sans Bengali,36,&H00FFFFFF,&H00000000,&H00000000,1,2,0,2,10,10,30,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""

ass_path = "/content/drive/MyDrive/subtitles/generated_clean_split.ass"

with open(ass_path, "w", encoding="utf-8") as f:
    f.write(ass_header)
    for start, end, text in translated_segments:
        f.write(f"Dialogue: 0,{format_ass_time(start)},{format_ass_time(end)},Default,,0,0,0,,{text}\n")

print("✅ সাবটাইটেল ফাইল তৈরি হয়েছে:", ass_path)

তোমার বাংলা ডায়লগগুলো লিখো (এক লাইনে বা একাধিক লাইনে):
হে ঈশ্বর, ফিলিস্তিনে তাদের সাহায্য করুন এবং তাদের সকল উদ্বেগ থেকে মুক্তি দিন, সকল দুঃখ থেকে মুক্তির পথ দিন এবং সকল দুঃখ থেকে সুস্থতা দান করুন।
ভিডিওর মোট দৈর্ঘ্য কত সেকেন্ড? (শুধু সংখ্যা):
9
✅ সাবটাইটেল ফাইল তৈরি হয়েছে: /content/drive/MyDrive/subtitles/generated_clean_split.ass


In [21]:
# Step 6: ভিডিওতে বাংলা সাবটাইটেল বার্ন করা
!ffmpeg -i "/content/drive/MyDrive/Videos/yd.mp4" -vf "ass=/content/drive/MyDrive/subtitles/ydas.ass" -c:a copy "/content/drive/MyDrive/Videos/yd_output.mp4"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab